In [ ]:
import grpc
import cv2
import numpy as np
from tensorflow_serving.apis import prediction_service_pb2_grpc
from tensorflow_serving.apis import get_model_metadata_pb2
from tensorflow_serving.apis import predict_pb2
from tensorflow.contrib.util import make_tensor_proto

fruit_list = ["Apricot", "Avocado", "Banana", "Clementine", "Cocos", "Kiwi", "Lemon", "Limes", 
              "Mandarine", "Orange", "Peach", "Pineapple", "Plum", "Pomegranate", "Raspberry", "Strawberry"]

In [ ]:
"""
Die Verbindungsdaten zum TensorFlow-Serving ModelServer
"""
channel = grpc.insecure_channel("from-keras-to-production-workshop_serving_1:8500")
stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)
stub?

In [ ]:
status = get_model_metadata_pb2.GetModelMetadataRequest()
status.model_spec.name = 'fruits'
status.model_spec.version.value = 4
status.metadata_field.append("signature_def")
response = stub.GetModelMetadata(status, 10)
response?

In [ ]:
request = predict_pb2.PredictRequest()
request.model_spec.name = 'fruits'
request.model_spec.version.value = 4
img = cv2.imread("../01-deeplearning/test_images/Banana/Banana_wiki.jpg").astype(np.float32)
img = cv2.resize(img, (20, 20))
img = img / 255
x, y, c = img.shape
proto = make_tensor_proto(img, shape=[1, x, y, c])
request.inputs['input'].CopyFrom(proto)
result = stub.Predict(request)  # 10 secs timeout
result?

In [ ]:
prediction = result.outputs['prediction'].float_val
max_value = max(prediction)
max_index = np.argmax(prediction)
print(fruit_list[max_index], max_value)